# 1. Receptor Generation

Autodock Vina requires the receptor files to be generated. This is done with the `Meeko` program, the companion llibrary to Autodock Vina. The center and box size of the receptor will have to be passed. This is fixed as a box surrounding the co-ligand. The hydrogens are added to the protein at the right pH with `PDB2PQR`. 

In [9]:
## define binary
mk_prepapre_receptor = 'mk_prepare_receptor.py'
pdb2pqr = 'pdb2pqr'

Download the PDB file:

In [10]:
!wget https://files.rcsb.org/view/1PPB.pdb

--2025-10-25 15:04:36--  https://files.rcsb.org/view/1PPB.pdb
Resolving files.rcsb.org (files.rcsb.org)... 18.64.247.95, 18.64.247.109, 18.64.247.122, ...
Connecting to files.rcsb.org (files.rcsb.org)|18.64.247.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281637 (275K) [text/plain]
Saving to: ‘1PPB.pdb’

1PPB.pdb            100%[===================>] 275.04K   353KB/s    in 0.8s    

2025-10-25 15:04:37 (353 KB/s) - ‘1PPB.pdb’ saved [281637/281637]



Now, the protein is cleaned (all non-protein residues are removed) and written to file `1ppb_prt.pdb`. The coordinates of the co-ligand are also read.

In [11]:
import numpy as np
from Bio.PDB import PDBParser, PDBIO, is_aa

parser = PDBParser()
structure = parser.get_structure("1ppb", "1PPB.pdb")

coords = []
    
for model in structure:
    for chain in model:
        residues_to_delete = []
        for residue in chain:
            resname = residue.get_resname()
            if resname == '0G6':
                for atom in residue:
                    atom_name = atom.get_name()
                    atom_coordinates = atom.get_vector() # Returns a Vector object
                    coords.append(atom_coordinates.get_array())
            
            if not is_aa(residue, standard=True):  # Non-standard amino acids, ligands, water, etc.
                residues_to_delete.append(residue)
        for residue in residues_to_delete:
            chain.detach_child(residue.id)

# Save the filtered structure
io = PDBIO()
io.set_structure(structure)
io.save("1ppb_prt.pdb")

coords = np.array(coords)

/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 3027.
  warnings.warn(
/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 3074.
  warnings.warn(


Hydrogens are now added to `1ppb_prt.pdb` at physiological pH.

In [12]:
!{pdb2pqr} 1ppb_prt.pdb 1ppb_clean.pqr --pH 7.4

/Users/vbf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: 1ppb_prt.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 295 residues and 2380 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolec

As the name suggestes, the program `pdb2pqr` outputs the protein in the `.pqr` format. This is mostly compatible with the PDB format, and most PDB readers should be able to read the PQR format as well. But, `mk_prepapre_receptor.py` breaks with this format. So, let us just load it in BioPython and right it back to a file.

In [13]:
parser = PDBParser()
structure = parser.get_structure("1ppb", "1ppb_clean.pqr")

# Save the filtered structure
io = PDBIO()
io.set_structure(structure)
io.save("1ppb_clean.pdb")

/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/PDBParser.py:236: PDBConstructionWarning: Negative occupancy in one or more atoms
  warnings.warn(
/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 1.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 2.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/Users/vbf/Library/Python/3.9/lib/python/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Us

The center of the ligand and a bounding box around the ligand (with a 6 Å) padding is also calculated.

In [14]:
center_coords = coords.mean(axis=0)
center = " ".join([f"{i:.2f}" for i in center_coords])
center

'5.04 17.14 18.38'

In [15]:
box_coords = (np.max(coords, axis=0)-np.min(coords, axis=0)+6).round()
box = " ".join([f"{int(i)}" for i in box_coords])
box

'14 11 18'

The center and the box coordinates are used to generate the receptor, centered on the co-ligand.

In [16]:
cmd = f"{mk_prepapre_receptor} -i 1ppb_clean.pdb -o 1ppb_receptor -p -v --box_size {box} --box_center {center}"
print(cmd)
!{cmd}

/Users/vbf/Library/Python/3.9/bin/mk_prepare_receptor.py -i 1ppb_clean.pdb -o 1ppb_receptor -p -v --box_size 14 11 18 --box_center 5.04 17.14 18.38
@> 4725 atoms and 1 coordinate set(s) were parsed in 0.02s.

Files written:
  1ppb_receptor.pdbqt <-- static (i.e., rigid) receptor input file
1ppb_receptor.box.txt <-- Vina-style box dimension file
1ppb_receptor.box.pdb <-- PDB file to visualize the grid box
